In [11]:
import pickle
from collections import Counter
from tqdm import tqdm
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
import re

In [2]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='tok'

In [3]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,_,_=pickle.load(f)

CPU times: user 4.99 s, sys: 1.23 s, total: 6.22 s
Wall time: 6.2 s


## Without text processing

#### Texts

In [6]:
lengths=[len(x['text']) for x in train]
pd.DataFrame(lengths).describe()

,0
count,551399.000000
mean,95.317418
std,94.979352
min,3.000000
25%,40.000000
50%,67.000000
75%,116.000000
max,4130.000000


In [ ]:
#### Summary

In [7]:
lengths=[len(x['summary']) for x in train]
pd.DataFrame(lengths).describe()

,0
count,551399.000000
mean,4.896286
std,3.173677
min,1.000000
25%,3.000000
50%,4.000000
75%,6.000000
max,79.000000


## After text processing
(for sequence model padding decision)

In [12]:
ALLOWED_SPECIAL=tuple(['?','!',':(', ':)', ':D',':-)',':-D',':\'(',':/',':-/','<3',':-P',':P'])

In [13]:
rx_special= re.compile("|".join(re.escape(s) for s in ALLOWED_SPECIAL))

def word_filter(word):
    if word in STOP_WORDS:
        return False
    if not word.isalpha():
        if not rx_special.findall(word):
            return False
    else:
        if len(word)<3:
            return False
    return True

def preprocess_text(text):
    return [x.lower() for x in text if word_filter(x.lower())]

def preprocess_texts(dataset,text_keys=['summary','text']):
    for sample in tqdm(dataset):
        for key in text_keys:
            sample[key]=preprocess_text(sample[key])
    return dataset

#### Texts

In [14]:
lengths=[len(x['text']) for x in preprocess_texts(train)]
pd.DataFrame(lengths).describe()

100%|██████████| 551399/551399 [00:18<00:00, 29221.89it/s]


,0
count,551399.000000
mean,34.747887
std,34.744651
min,0.000000
25%,15.000000
50%,24.000000
75%,42.000000
max,1744.000000


#### Summaries

In [15]:
lengths=[len(x['summary']) for x in preprocess_texts(train)]
pd.DataFrame(lengths).describe()

100%|██████████| 551399/551399 [00:09<00:00, 56009.10it/s]


,0
count,551399.000000
mean,3.033848
std,1.694317
min,0.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,74.000000
